In [1]:
import pandas as pd
import re
import pickle
import string

In [2]:
def repair_caption(caption) :
    caption = caption.replace(',', ' , ')
    caption = re.sub(' +', ' ', caption)
    caption = caption.replace(' ,', ',')

    return caption

In [3]:
train_csv = None
caption_list_audiocaps = []
caption_list_clotho = []

train_csv = pd.read_csv('train.csv')

captions = train_csv['caption'].to_list()
for caption in captions:

    edited_caption = repair_caption(caption)
    caption_list_audiocaps.append(caption)

train_csv = pd.read_csv('clotho_captions_development.csv')

for i in range(5) :
    col_str = 'caption_' + str(i + 1)
    
    captions = train_csv[col_str].to_list()
    for caption in captions:
        edited_caption = repair_caption(caption)
        caption_list_clotho.append(caption)

In [4]:
def make_dictionary(caption_list) :

    word_dictionary = {}

    for caption in caption_list :
        caption = caption.lower()
        
        # 쉼표를 '안녕,하세요'와 같이 쓰는 사례가 많더라
        # 얘들을 다 교정해준다 
        # 교정========================================
        caption = caption.replace(',', ' , ') 
        # 공백 줄이기
        caption = re.sub(' +', ' ', caption)
        caption = caption.replace(' ,', ',')
        # 교정========================================

        # For ACT Tokenizer
        # caption = re.sub(r'\s([,.!?;:"](?:\s|$))', r'\1', caption).replace('  ', ' ')
        # caption = re.sub('[,.!?;:\"]', ' ', caption).replace('  ', ' ')

        # For Clotho tokenizer
        caption = caption.translate(str.maketrans('', '', string.punctuation))

        # For Custom Tokenizer1
        # caption = re.sub(r'[.]', '', caption)
        # caption += '.'

        # For Custom Tokenizer2
        # caption = re.sub(r'[.]', '', caption)

        word_list = caption.strip().split(' ')

        for word in word_list : 
            
            # Custom Tokenizer2의 경우 사용================================
            # if word in word_dictionary: 
            #     if word[-1] == ',' :
            #         temp_word_wo_rest = word[:-1] # 쉼표 있으면 쉼표 제거
            #         word_dictionary[temp_word_wo_rest] += 1
            #         word_dictionary[','] += 1 # 쉼표를 따로 추가
            #     else :
            #         word_dictionary[word] += 1
            # else : 
            #     if word[-1] == ',' :
            #         temp_word_wo_rest = word[:-1] # 쉼표 있으면 쉼표 제거
            #         word_dictionary[temp_word_wo_rest] = 1
            #         word_dictionary[','] = 1
            #     else :
            #         word_dictionary[word] = 1
            # Custom Tokenizer2의 경우 사용================================

            if word in word_dictionary: 
                word_dictionary[word] += 1
            else : 
                word_dictionary[word] = 1
    
    word_dictionary_sorted = sorted(word_dictionary.items(), key = lambda item: item[1], reverse = True)

    word_Dict = []

    # 단어만 담기
    # 그러면 등장 빈도수가 높은 단어부터 나열되는 딕셔너리가 완성됨
    for key, value in word_dictionary_sorted :
        word_Dict.append(key)

    word_Dict.insert(0, '!')
    word_Dict.insert(13, '<eos>')
    word_Dict.append('<unk>')

    return word_Dict

In [5]:
word_Dict_audiocaps = make_dictionary(caption_list_audiocaps)
word_Dict_clotho = make_dictionary(caption_list_clotho)


In [12]:
count = 0
for word in word_Dict_clotho :
    try :
        word_idx = word_Dict_audiocaps.index(word)
    except :
        print(word, "is not in Audiocaps vocab!")
        count +=1

print()
print(str(count), "word is not in AudioCaps Vocab!")
print("AudioCaps has", str(len(word_Dict_audiocaps)), "vocab")

book is not in Audiocaps vocab!
stairs is not in Audiocaps vocab!
hall is not in Audiocaps vocab!
gutter is not in Audiocaps vocab!
travelling is not in Audiocaps vocab!
restaurant is not in Audiocaps vocab!
stations is not in Audiocaps vocab!
wearing is not in Audiocaps vocab!
hallway is not in Audiocaps vocab!
heels is not in Audiocaps vocab!
porch is not in Audiocaps vocab!
playground is not in Audiocaps vocab!
thin is not in Audiocaps vocab!
market is not in Audiocaps vocab!
any is not in Audiocaps vocab!
flipped is not in Audiocaps vocab!
kettle is not in Audiocaps vocab!
drove is not in Audiocaps vocab!
printing is not in Audiocaps vocab!
approached is not in Audiocaps vocab!
cicadas is not in Audiocaps vocab!
create is not in Audiocaps vocab!
branches is not in Audiocaps vocab!
elevator is not in Audiocaps vocab!
coin is not in Audiocaps vocab!
marker is not in Audiocaps vocab!
reverb is not in Audiocaps vocab!
erratic is not in Audiocaps vocab!
office is not in Audiocaps vocab!

In [13]:
count = 0
for word in word_Dict_audiocaps :
    try :
        word_idx = word_Dict_clotho.index(word)
    except :
        print(word, "is not in Clotho vocab!")
        count +=1

print()
print(str(count), "word is not in Clotho Vocab!")
print("AudioCaps has", str(len(word_Dict_clotho)), "vocab")

pigeons is not in Clotho vocab!
snores is not in Clotho vocab!
language is not in Clotho vocab!
foreign is not in Clotho vocab!
males is not in Clotho vocab!
triggered is not in Clotho vocab!
whoosh is not in Clotho vocab!
oinks is not in Clotho vocab!
females is not in Clotho vocab!
meow is not in Clotho vocab!
gunshot is not in Clotho vocab!
burp is not in Clotho vocab!
mans is not in Clotho vocab!
monologue is not in Clotho vocab!
speeches is not in Clotho vocab!
muffling is not in Clotho vocab!
highpitched is not in Clotho vocab!
delivering is not in Clotho vocab!
skidding is not in Clotho vocab!
clipclop is not in Clotho vocab!
neighs is not in Clotho vocab!
clops is not in Clotho vocab!
speedboat is not in Clotho vocab!
narrates is not in Clotho vocab!
clipclops is not in Clotho vocab!
whooshes is not in Clotho vocab!
delivers is not in Clotho vocab!
oink is not in Clotho vocab!
sputtering is not in Clotho vocab!
narrating is not in Clotho vocab!
yipping is not in Clotho vocab!
c

In [18]:
file_name = None
if Dataset == 'AudioCaps' :
    file_name = 'AudioCaps_vocabulary_'
elif Dataset == 'Clotho' :
    file_name = 'Clotho_vocabulary_'

file_name += (str(len(word_Dict)) + '.pickle')

with open(file_name, 'wb') as f:
    pickle.dump(word_Dict, f, pickle.HIGHEST_PROTOCOL)

In [19]:
len(word_Dict)

4368

In [24]:
for i in range(len(word_Dict)) :
    if word_Dict[i] != word_Dict_act[i] :
        print(word_Dict[i], word_Dict_act[i])

walkietalkie walkie-talkie


In [26]:
word_Dict.index('walkietalkie')

3650

In [27]:
word_Dict_act.index('walkie-talkie')

3650